Импорт нужных либ

In [33]:
import numpy as np
import matplotlib.pyplot as plt
from sympy import *
%matplotlib inline

# Лабораторная работа №1. Задание 1. Вариант 25
Решается обыкновенное дифференциальное уравнение
$ U''(x) = f(x, U, U'), x \in [a; b] $
С краевыми (граничными) условиями
$ U(a) = A; $
$ \mu_1U(b) + \nu_1U'(b) = B $
Метод решения уравнения $ \varphi(\alpha) = 0 $ - метод половинного деления
Направление интегрирования: $ a \implies b $

Входные параметры

In [34]:
# Границы отрезка, на котором ищется решение:
a, b = -1, 1
# Число отрезков разбиения
N = 50
# Точность решения нелинейного уравнения
epsilon = 1e-3
# Предельное число итераций
K = 25
# Начальное значение пристрелочного параметра
alpha = 0
# Использовать коэффициенты A и B или mu1 и nu1
# Константы краевых уравнений
A, B = -2, 1
mu1 = 1
nu1 = 1

In [35]:
# Шаг
h = (b-a) / N
flag_IER = True
iterations_count = 0 # Для итераций в МПД

x_array = Symbol('x')
u = Symbol('u')
w = Symbol('w')
# Задание функции
f_exact_solution = x_array ** 2
exact_solution = lambdify(x_array, f_exact_solution)

fd_exact_solution = diff(f_exact_solution, x_array)
derivative_exact_solution = lambdify(x_array, fd_exact_solution)

# Всякие функции для работы лабы

In [36]:
# Разные функции для работы программы
def x_i():
        x_array = np.zeros(N + 1)
        for i in range(N + 1):
            x_array[i] = float(a + i * h)
        return x_array

# Функция первоначальной задачи Коши
f1 = lambda x, u, w: w

# Вторая производная
f2d_exact_solution = diff(fd_exact_solution, x_array)
f2 = lambdify([x_array, u, w], f2d_exact_solution)

# Метод Рунге Кутта 4 порядка:
def RungeKut(input_u, input_w, input_alpha):
    input_u[0] = A # u
    input_w[0] = input_alpha # w
    k1 = np.zeros(2)
    k2 = np.zeros(2)

    for i in range(N):
        k1[0] = (h * f1(x_array[i], input_u[i], input_w[i]))
        k1[1] = (h * f2(x_array[i], input_u[i], input_w[i]))

        k2[0] = (h * f1(x_array[i] + h, input_u[i] + k1[0], input_w[i] + k1[1]))
        k2[1] = (h * f2(x_array[i] + h, input_u[i] + k1[0], input_w[i] + k1[1]))

        input_u[i + 1] = (input_u[i] + 0.5 * (k1[0] + k2[0]))
        input_w[i + 1] = (input_w[i] + 0.5 * (k1[1] + k2[1]))
    return input_u, input_w

def fiAlpha(input_alpha, input_u_array, input_w_array):
    # В тмпу хранятся приближенные значения точного решения функции
    # В тмпв знач. первой производной функции
    tmp_u, tmp_w = RungeKut(input_u_array, input_w_array, input_alpha)
    # Значение функции на границе - В, где В - значение производной функции с права
    fi_alpha_value = tmp_u[N] - B
    return fi_alpha_value

def different_sign(lvalue, rvalue):
    return lvalue * rvalue < 0

 # (страница 14 пункт 3, можно сделать свой алгоритм, мне скинули такой)
def find_alpha2_value(alpha1, input_u_array, input_w_array):
    phi_alpha1 = fiAlpha(alpha1, input_u_array, input_w_array)
    delta = 1.0
    alpha_2 = alpha1 + delta
    phi_alpha2 = fiAlpha(alpha_2, input_u_array, input_w_array)
    flag_reversed = False
    while not different_sign(phi_alpha1, phi_alpha2):
        alpha_2 = alpha1 + delta
        phi_alpha2 = fiAlpha(alpha_2, input_u_array, input_w_array)
        if phi_alpha1 < 0:
            if not (phi_alpha2 > phi_alpha1 and flag_reversed):
                delta += 0.2
            elif phi_alpha2 > phi_alpha1 and flag_reversed:
                delta -= 0.2
            elif phi_alpha2 < phi_alpha1:
                delta = delta * (-1)
                flag_reversed = True
        if phi_alpha1 > 0:
            if phi_alpha2 > phi_alpha1:
                delta = delta * (-1)
                flag_reversed = True
            else:
                if not phi_alpha2 < phi_alpha1 and flag_reversed:
                    delta += 0.2
                if phi_alpha2 < phi_alpha1 and flag_reversed:
                    delta -= 0.2
    return alpha_2

# МПД
def bisection(neg_point, pos_point, input_epsilon, input_u_array, input_w_array):
    global flag_IER
    global iterations_count
    iterations_count += 1
    if iterations_count > K:
        print('IER = 1:Количество итераций слишком велико')
        flag_IER = False
        return None
    average_point = (pos_point + neg_point) / 2  # Средняя точка между концами интервала
    if abs(neg_point - pos_point) < input_epsilon:  # Основное условие для нахождения нуля функции
        return average_point
    test_value = fiAlpha(average_point, input_u_array, input_w_array)  # Значение функции в средней точке
    if test_value > 0:
        return bisection(neg_point, average_point, input_epsilon, input_u_array, input_w_array)
    elif test_value < 0:
        return bisection(average_point, pos_point, input_epsilon, input_u_array, input_w_array)
    return average_point

# Код алгоритма

In [37]:
if N < 0 or K <= 0 or epsilon < 0:
    flag_IER = False
    IER = 2
    print('IER', IER, ':Неправильные значения')
while flag_IER:
    x_array = x_i()
    u_array = np.zeros(N + 1)
    w_array = np.zeros(N + 1)
    u_array, w_array = RungeKut(u_array, w_array, alpha)

    alpha2 = find_alpha2_value(alpha, u_array, w_array)

    result = bisection(alpha, alpha2, epsilon, u_array, w_array)
    print('Фи в альфа*', fiAlpha(result, u_array, w_array))
    print(result)

    # Заголовок вывода
    if not flag_IER:
        break
    print('EIR = 0:Безошибочная работа программы')
    print('Количество итераций', iterations_count)
    print('Конечное значение параметра пристрелки', result)
    str0 = "   x"
    str1 = "     y"
    str2 = "    deltaU"
    str3 = "  y'"
    str4 = "        deltaU'"
    print('{0:8s}{1:14s}{2:14s}{3:14s}{4:14s}'.format(str0, str1, str2, str3, str4))

    for i in np.arange(0, N + 1):
        xi = x_array[i]
        y1i = exact_solution(x_array[i])
        deltau = abs(exact_solution(x_array[i]) - u_array[i])
        y_ = derivative_exact_solution(x_array[i])
        deltau_ = abs(derivative_exact_solution(x_array[i]) - w_array[i])
        print('{0:8.5f}{1:14.5e}{2:14.5e}{3:14.5e}{4:14.5e}'.format(xi, y1i, deltau, y_, deltau_))

    flag_IER = False

Фи в альфа* 0.9990234375000013
-0.00048828125
EIR = 0:Безошибочная работа программы
Количество итераций 11
Конечное значение параметра пристрелки -0.00048828125
   x         y            deltaU      y'                  deltaU'
-1.00000   1.00000e+00   3.00000e+00  -2.00000e+00   1.99951e+00
-0.96000   9.21600e-01   2.92002e+00  -1.92000e+00   1.99951e+00
-0.92000   8.46400e-01   2.84004e+00  -1.84000e+00   1.99951e+00
-0.88000   7.74400e-01   2.76006e+00  -1.76000e+00   1.99951e+00
-0.84000   7.05600e-01   2.68008e+00  -1.68000e+00   1.99951e+00
-0.80000   6.40000e-01   2.60010e+00  -1.60000e+00   1.99951e+00
-0.76000   5.77600e-01   2.52012e+00  -1.52000e+00   1.99951e+00
-0.72000   5.18400e-01   2.44014e+00  -1.44000e+00   1.99951e+00
-0.68000   4.62400e-01   2.36016e+00  -1.36000e+00   1.99951e+00
-0.64000   4.09600e-01   2.28018e+00  -1.28000e+00   1.99951e+00
-0.60000   3.60000e-01   2.20020e+00  -1.20000e+00   1.99951e+00
-0.56000   3.13600e-01   2.12021e+00  -1.12000e+00   1.999

In [38]:
# y = np.zeros(N + 1)
# deltau = np.zeros(N + 1)
# y_ = np.zeros(N + 1)
# deltau_ = np.zeros(N + 1)
# for i in np.arange(0, N + 1):
#         y[i] = exact_solution(x[i])
#         deltau[i] = abs(exact_solution(x[i]) - u[i])
#         y_[i] = derivative_exact_solution(x[i])
#         deltau_[i] = abs(derivative_exact_solution(x[i]) - v[i])
# plt.figure(figsize=(10,6))
# plt.plot(
#     x, y,
#     x, y_
# )
# plt.legend([
#     'Y',
#     "Y'"
# ])
# plt.grid(true)
# plt.title("Функции $Y$ и $Y'$")
# plt.xlabel("X")
# plt.ylabel("Y")
#
# plt.figure(figsize=(10,6))
# plt.plot(
#     x, deltau
# )
# plt.legend([
#     "$\delta$U"
# ])
# plt.grid(true)
# plt.title("Абсолютная погрешность для $Y$")
# plt.xlabel("X")
# plt.ylabel("Y")
#
# plt.figure(figsize=(10,6))
# plt.plot(
#     x, deltau_
# )
# plt.legend([
#     "$\delta$U'"
# ])
# plt.grid(true)
# plt.title("Абсолютная погрешность для $Y'$")
# plt.xlabel("X")
# plt.ylabel("Y")

# Задание 2. Вариант 6
Решается обыкновенное дифференциальное уравнение
$ U'''(x) = f(x, U, U', U''), x \in [a; b] $
С краевыми (граничными) условиями
$ U'(a) = A; $
$ U(b) = B, U''(b) = C $
Метод решения уравнения $ \varphi(\alpha) = 0 $ - метод секущих
Направление интегрирования: $ a \implies b $